In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import numpy as np
import scipy.integrate

import bokeh.io
import bokeh.plotting

import panel as pn

import colorcet

bokeh.io.output_notebook()
pn.extension()

# List of Species
# Nn = NF-kB in nucleus
# Im = IkBa mRNA
# I = IkBa protein
# Am = A20 mRNA
# A = A20 protein
# IRAK = Active IRAK level
# IRAKi = Inactive IRAK level
# IKKa = IKK active
# IKKi = IKK inactive
# Iem = IkBe mRNA level
# Ie = IkBe proptein level
# NI = NFkB::IkBe level
# TNF = TNF level
# IL1 = IL1-B level
# LPS = LPS level
# Dur = The duration that NF-kB has been above the threshold

# Species w/ fast equilibirum assumption
# R = The fraction of active receptor

# Other
# TNF = TNF concentration

def NFkB_cascade(ini, t, par, Dur_Thr):
    
    # Unpack species
    Nn, Im, I, Am, A, IRAKlps, IRAKilps, IRAKil1, IRAKiil1, IKKa, IKKi, TNF, IL1, LPS = ini
    
    # Unpack parameters
    rNim, KIc, rIim, KIn, trI, KN, dIm, tlI, aIKK, trA, dAm, tlA, dA, aR, Clig, Ca20tnf, Ca20lps, aIRAK, dIRAK, dIRAKi, mu, beta, dL = par
    # List of Parameters
    # rNim = NF-kB imporation rate
    # KIc = dissociation constant for IkBa binding to NF-kB (in cytosol)
    # rIim = IkBa imporation rate
    # KIn = dissociation constant for IkBa binding to NF-kB (in nucleus)
    # trI = transcription rate for Ikba mRNA
    # KN = Dissociation constant for NF-kB inducing mRNA
    # dIm = Degradation rate of IkBa mRNA
    # tlI = translation rate for Ikba
    # aIKK = IKK dependent degration rate of IkBa
    # dI = degradation rate of IkBa
    # trA = transcription rate of A20
    # dAm = degradation rate of A20 mRNA
    # tlA = translation rate of A20
    # dA = degradation rate of A20
    # aR = the rate that active receptor activates IKK
    # Clig = dissociation constant for ligand binding receptor
    # Ca20tnf = dissociation constant for A20 binding TNFR
    # Ca20il1 = dissociation constant for A20 binding IL1R
    # Ca20lps = dissociation constant for A20 binding TLR4
    # aIRAK = rate that active il1 or lps receptor inducing active IRAK
    # dIRAK = rate that IRAK becomes inactive
    # dIRAKi = rate that IRAK becomes inactive
    # mu = inactivation rate of IKK
    # beta = neutralization rate of IKKi
    # trIe = transcription rate of IkBe mRNA
    # dIem = degradation rate of IkBe mRNA
    # KDur = dissociation constant for NF-kB turned on duration affecting the IkBe expression
    # tlIe = translation rate of IkBe
    # dIe = degradation rate of IkBe
    # rNI = association rate between NFkB and IkBe
    # dNI = dissociation rate of NFkB::IkBe
    # dL = degradation rate of ligand
    KIc = KIc/100
    KIn = KIn/100
    
    # Compute dNn/dt or NFkB in nuc
    dNn_dt = rNim*(1-Nn)/(KIc+I) - rIim*(I*Nn)/(KIn+Nn)

    # Compute dIm/dt or IkBa mRNA change rate
    dIm_dt = trI*(Nn**4)/(KN**4 + Nn**4) - dIm*Im
    
    # Compute dI/dt or IkBa protein change rate
    dI_dt = tlI*Im - aIKK*IKKa*(1-Nn)*I/(KIc+I)
    
    # Compute dAm/dt or A20 mRNA change rate
    dAm_dt = trA*(Nn**4)/(KN**4 + Nn**4) - dAm*Am
    
    # Compute dA/dt or A20 protein change rate
    dA_dt = tlA*Am - dA*A
    
    # Calculate Current Active TNF Receptor level
    Rtnf = (TNF**3)/(TNF**3 + Clig**3)
    
    # Calculate Current Active IL1 Receptor level
    Ril1 = (IL1**3)/(IL1**3 + Clig**3)
    
    # Calculate Current Active LPS Receptor level
    Rlps = (LPS**3)/(LPS**3 + Clig**3)
    
    # Active LPS IRAK dynamics
    dIRAKlps_dt = aIRAK*Rlps*(1-IRAKlps-IRAKilps-IRAKil1-IRAKiil1) - dIRAK*IRAKlps**2
    
    # InActive LPS IRAK dynamics
    dIRAKilps_dt = dIRAK*IRAKlps**2 - dIRAKi*IRAKilps
    
    # Active IRAK IL1 dynamics
    dIRAKil1_dt = aIRAK*Ril1*(1-IRAKlps-IRAKilps-IRAKil1-IRAKiil1) - dIRAK*IRAKil1**2
    
    # InActive IRAK IL1 dynamics
    dIRAKiil1_dt = dIRAK*IRAKil1**2 - dIRAKi*IRAKiil1
    
    # IKKa dynamics
    dIKKa_dt = (1-IKKa-IKKi)*aR*(Rtnf*(Ca20tnf**3)/(Ca20tnf**3 + A**3) + IRAKlps*(Ca20lps**3)/(Ca20lps**3 + A**3) + IRAKil1*(Ca20lps**3)/(Ca20lps**3 + A**3)) - mu*IKKa**2
    
    # IKKi dynamics
    dIKKi_dt = mu*IKKa**2 - beta*IKKi
    
    # TNF dynamics
    dTNF_dt = -dL*TNF
    
    # IL1 dynamics
    dIL1_dt = -dL*IL1
    
    # LPS dynamics
    dLPS_dt = -dL*LPS
    
    # Return the result as a NumPy array
    return np.array([dNn_dt, dIm_dt, dI_dt, dAm_dt, dA_dt, dIRAKlps_dt, dIRAKilps_dt, dIRAKil1_dt, dIRAKiil1_dt, dIKKa_dt, dIKKi_dt, dTNF_dt, dIL1_dt, dLPS_dt])


Loading BokehJS ...

In [3]:
from bokeh.io import export_svg

# Set up color palette for this notebook
colors = colorcet.b_glasbey_category10

# Parameters
# par_0 = np.array([11.3, 3.5, 1.09, 2.90, 59.5, 0.71, 2.00, 14.4, 63.0, 0.4,
#                    5.0, 1.0, 15.0, 0.10, 1.00, 1.00, 5.0,  5.0,  5.0,  0.1, 
#                    0.1, 0.1, 10.0, 2.00, 59.5, 2.00, 40,   14.4, 0.4,  0.5,
#                    0.5, 1.00])
# ^ in the order of rNim,  KIc,    rIim, KIn,  trI,  KN,   dIm,     tlI,     aIKK,    dI,
#                   trA,   dAm,    tlA,  dA,   aR,   Ctnf, Ca20tnf, Ca20il1, Ca20lps, aIRAK,
#                   dIRAK, dIRAKi, mu,   beta, trIe, dIem, KDur,    tlIe,    dIe,   rNI,
#                   dNI,    dL

rNim_slider = pn.widgets.FloatSlider(
    name="rNim_p", start=-5, end=5, step=.1, value=0
)
KIc_slider = pn.widgets.FloatSlider(
    name="KIc_p", start=-5, end=5, step=.1, value=0
)
rIim_slider = pn.widgets.FloatSlider(
    name="rIim_p", start=-5, end=5, step=.1, value=0
)
KIn_slider = pn.widgets.FloatSlider(
    name="KIn_p", start=-5, end=5, step=.1, value=0
)
trI_slider = pn.widgets.FloatSlider(
    name="trI_p", start=-5, end=5, step=.1, value=0
)
KN_slider = pn.widgets.FloatSlider(
    name="KN_p", start=-5, end=5, step=.1, value=0
)
dIm_slider = pn.widgets.FloatSlider(
    name="dIm_p", start=-5, end=5, step=.1, value=0
)
tlI_slider = pn.widgets.FloatSlider(
    name="tlI_p", start=-5, end=5, step=.1, value=0
)
aIKK_slider = pn.widgets.FloatSlider(
    name="aIKK_p", start=-5, end=5, step=.1, value=0
)
trA_slider = pn.widgets.FloatSlider(
    name="trA_p", start=-5, end=5, step=.1, value=0
)
dAm_slider = pn.widgets.FloatSlider(
    name="dAm_p", start=-5, end=5, step=.1, value=0
)
tlA_slider = pn.widgets.FloatSlider(
    name="tlA_p", start=-5, end=5, step=.1, value=0
)
dA_slider = pn.widgets.FloatSlider(
    name="dA_p", start=-5, end=5, step=.1, value=-2
)
aR_slider = pn.widgets.FloatSlider(
    name="aR_p", start=-5, end=5, step=.1, value=0
)
Clig_slider = pn.widgets.FloatSlider(
    name="Clig_p", start=-5, end=5, step=.1, value=-0.4
)
Ca20tnf_slider = pn.widgets.FloatSlider(
    name="Ca20tnf_p", start=-5, end=5, step=.1, value=0
)
Ca20lps_slider = pn.widgets.FloatSlider(
    name="Ca20lps_p", start=-5, end=5, step=.1, value=0
)
aIRAK_slider = pn.widgets.FloatSlider(
    name="aIRAK_p", start=-5, end=5, step=.1, value=0
)
dIRAK_slider = pn.widgets.FloatSlider(
    name="dIRAK_p", start=-5, end=5, step=.1, value=0
)
dIRAKi_slider = pn.widgets.FloatSlider(
    name="dIRAKi_p", start=-5, end=5, step=.1, value=0
)
mu_slider = pn.widgets.FloatSlider(
    name="mu_p", start=-5, end=5, step=.1, value=.5
)
beta_slider = pn.widgets.FloatSlider(
    name="beta_p", start=-5, end=5, step=.1, value=0
)
dL_slider = pn.widgets.FloatSlider(
    name="dL_p", start=-5, end=5, step=.1, value=0
)
t_max_slider = pn.widgets.FloatSlider(
    name="t_max", start=0.1, end=8, step=0.1, value=2
)
normalize_checkbox = pn.widgets.Checkbox(name='Normalize', value=False)

# Defining the initial condition
ini_0 = np.array([0.0, 0.0, 1.0, 0.0, 0.0, 0.0,     0.0,      0.0,     0.0,      0.0,   0.0,   1.0,  0.0,  0.0])
#                 Nn,  Im,  I,   Am,  A,   IRAKlps, IRAKilps, IRAKil1, IRAKiil1, IKKa,  IKKi,  TNF,  IL1,  LPS = ini
global ini

# Defining the default paraters
rNim_0 = 11.3;   KIc_0 = 3.5;       rIim_0 = 1.09; KIn_0 = 2.90;     trI_0 = 59.5;  KN_0 = 0.6;    dIm_0 = 2.00;    tlI_0 = 14.4;    aIKK_0 = 63.0*2;  dI_0 = 0.4
trA_0 = 5.0;     dAm_0 = 1.0;       tlA_0 = 15.0;  dA_0 = 0.10;      aR_0 = 4.00;   Clig_0 = 8.00; Ca20tnf_0 = 8.0; Ca20il1_0 = 8.0; Ca20lps_0 = 8.0;  aIRAK_0 = 63*4
dIRAK_0 = 200.0; dIRAKi_0 = 0.005;  mu_0 = 10.0*2; beta_0 = 2.00/10; trIe_0 = 59.5; dIem_0 = 2.0;  KDur_0 = 40;     tlIe_0 = 14.4;   dIe_0 = 0.4;      rNI_0 = 1.0
dNI_0 = 1.0;     dL_0 = 0.00

@pn.depends(
    rNim_slider.param.value, KIc_slider.param.value,  rIim_slider.param.value, KIn_slider.param.value, trI_slider.param.value, KN_slider.param.value,  dIm_slider.param.value,
    tlI_slider.param.value,  aIKK_slider.param.value, trA_slider.param.value,  dAm_slider.param.value, tlA_slider.param.value, dA_slider.param.value,  aR_slider.param.value,
    Clig_slider.param.value, Ca20tnf_slider.param.value, Ca20lps_slider.param.value, aIRAK_slider.param.value, dIRAK_slider.param.value, dIRAKi_slider.param.value,
    mu_slider.param.value, beta_slider.param.value, dL_slider.param.value,
    t_max_slider.param.value, normalize_checkbox.param.value,
    )
def initial_response_plot(
    rNim_p,  KIc_p,   rIim_p, KIn_p,  trI_p,  KN_p,      dIm_p,     tlI_p,     aIKK_p,    trA_p,
    dAm_p,   tlA_p,   dA_p,   aR_p,   Clig_p, Ca20tnf_p, Ca20lps_p, aIRAK_p,   dIRAK_p,   dIRAKi_p,
    mu_p,    beta_p,  dL_p,
    t_max,  normalize,
#     ini_0,
#     rNim_0,  KIc_0,    rIim_0, KIn_0,  trI_0,  KN_0,   dIm_0,     tlI_0,     aIKK_0,    dI_0,
#     trA_0,   dAm_0,    tlA_0,  dA_0,   aR_0,   Ctnf_0, Ca20tnf_0, Ca20il1_0, Ca20lps_0, aIRAK_0,
#     dIRAK_0, dIRAKi_0, mu_0,   beta_0, trIe_0, dIem_0, KDur_0,    tlIe_0,    dIe_0,     rNI_0,
#     dNI_0,   dL_0
    ):
    
#     rNim=rNim_0*(2**rNim_p)
#     KIc=KIc_0*(2**KIc_p)
#     rIim=rIim_0*(2**rIim_p)
#     KIn=KIn_0*(2**KIn_p)
#     trI=trI_0*(2**trI_p)
#     KN=KN_0*(2**KN_p)
#     dIm=dIm_0*(2**dIm_p)
#     tlI=tlI_0*(2**tlI_p)
#     aIKK=aIKK_0*(2**aIKK_p)
#     trA=trA_0*(2**trA_p)
#     dAm=dAm_0*(2**dAm_p)
#     tlA=tlA_0*(2**tlA_p)
#     dA=dA_0*(2**dA_p)
#     aR=aR_0*(2**aR_p)
#     Clig=Clig_0*(2**Clig_p)
#     Ca20tnf=Ca20tnf_0*(2**Ca20tnf_p)
#     Ca20lps=Ca20lps_0*(2**Ca20lps_p)
#     aIRAK=aIRAK_0*(2**aIRAK_p)
#     dIRAK=dIRAK_0*(2**dIRAK_p)
#     dIRAKi=dIRAKi_0*(2**dIRAKi_p)
#     mu=mu_0*(2**mu_p)
#     beta=beta_0*(2**beta_p)
#     dL=dL_0*(2**dL_p)
    
    rNim=rNim_0
    KIc=KIc_0
    rIim=rIim_0
    KIn=KIn_0
    trI=trI_0
    KN=KN_0
    dIm=dIm_0
    tlI=tlI_0
    aIKK=aIKK_0
    trA=trA_0
    dAm=dAm_0
    tlA=tlA_0
    dA=dA_0
    aR=aR_0
    Clig=Clig_0
    Ca20tnf=Ca20tnf_0
    Ca20lps=Ca20lps_0
    aIRAK=aIRAK_0
    dIRAK=dIRAK_0
    dIRAKi=dIRAKi_0
    mu=mu_0
    beta=beta_0
    dL=dL_0
    
    par = np.array([rNim, KIc,  rIim, KIn,  trI, KN,  dIm,     tlI,     aIKK,   trA,
                    dAm,  tlA,  dA,   aR,   1,   8,   8, aIRAK,   dIRAK,  dIRAKi,
                    mu,   beta, 2])
    # Calculating Initial Condition
    args = (par, 1)
    t = np.linspace(0, 48, 1001)
    yz = scipy.integrate.odeint(NFkB_cascade, ini_0, t, args=args)
    Nn, Im, I, Am, A, IRAKlps, IRAKilps, IRAKil1, IRAKiil1, IKKa, IKKi, TNF, IL1, LPS = yz.transpose()
    global ini
    ini = yz[1000,:]
    ini[4] = .01
    
    # Normalize Y and Z responses
#     if normalize:
#         Nn /= Nn.max()
#         Im /= Im.max()
#         I /= I.max()
#         Am /= Am.max()
#         A /= A.max()
#         IRAK /= IRAK.max()
#         IRAKi /= IRAKi.max()
#         IKKa /= IKKa.max()
#         IKKi /= IKKi.max()
#         Iem /= Iem.max()
#         Ie /= Ie.max()
#         NI /= NI.max()
#         TNF /= TNF.max()
#         IL1 /= IL1.max()
#         LPS /= LPS.max()
#         Dur /= Dur.max()
    
    # Set up plot
#     p1 = bokeh.plotting.figure(
#          frame_width=700,
#          frame_height=100,
#          x_axis_label="Time (h)",
#          y_axis_label=f"{'Normalized ' if normalize else ''}Output",
#     )
    
#     # Populate glyphs
#     p1.line(t, Nn, line_width=2, color=colors[0], legend_label="NF-kBnuc")
#     p1.line(t, A,  line_width=2, color=colors[1], legend_label="Downstream Feedback")
#     p1.line(t, IRAKlps+IRAKil1, line_width=2, color=colors[2], legend_label="Active IRAK")
    
#     # Place the legend
#     p1.legend.location = "top_right"
    
    return

@pn.depends(
    rNim_slider.param.value, KIc_slider.param.value,     rIim_slider.param.value, KIn_slider.param.value, trI_slider.param.value, KN_slider.param.value,  dIm_slider.param.value,
    tlI_slider.param.value,  aIKK_slider.param.value,    trA_slider.param.value,  dAm_slider.param.value, tlA_slider.param.value, dA_slider.param.value,  aR_slider.param.value,
    Clig_slider.param.value, Ca20tnf_slider.param.value, Ca20lps_slider.param.value, aIRAK_slider.param.value, dIRAK_slider.param.value, dIRAKi_slider.param.value,
    mu_slider.param.value, beta_slider.param.value, dL_slider.param.value,
    t_max_slider.param.value, normalize_checkbox.param.value,
    )
def LPS_IL1_response_plot(
    rNim_p,  KIc_p,   rIim_p, KIn_p,  trI_p,  KN_p,      dIm_p,     tlI_p,     aIKK_p,    trA_p,
    dAm_p,   tlA_p,   dA_p,   aR_p,   Clig_p, Ca20tnf_p, Ca20lps_p, aIRAK_p,   dIRAK_p,   dIRAKi_p,
    mu_p,    beta_p,  dL_p,
    t_max,  normalize,
#     rNim_0,  KIc_0,    rIim_0, KIn_0,  trI_0,  KN_0,   dIm_0,     tlI_0,     aIKK_0,    dI_0,
#     trA_0,   dAm_0,    tlA_0,  dA_0,   aR_0,   Ctnf_0, Ca20tnf_0, Ca20il1_0, Ca20lps_0, aIRAK_0,
#     dIRAK_0, dIRAKi_0, mu_0,   beta_0, trIe_0, dIem_0, KDur_0,    tlIe_0,    dIe_0,     rNI_0,
#     dNI_0,   dL_0
    ):
    
    rNim=rNim_0*(2**rNim_p)
    KIc =KIc_0*(2**KIc_p)
    rIim=rIim_0*(2**rIim_p)
    KIn =KIn_0*(2**KIn_p)
    trI =trI_0*(2**trI_p)
    KN  =KN_0*(2**KN_p)
    dIm =dIm_0*(2**dIm_p)
    tlI =tlI_0*(2**tlI_p)
    aIKK=aIKK_0*(2**aIKK_p)
    trA =trA_0*(2**trA_p)
    dAm =dAm_0*(2**dAm_p)
    tlA =tlA_0*(2**tlA_p)
    dA  =dA_0*(2**dA_p)
    aR  =aR_0*(2**aR_p)
    Clig=Clig_0*(2**Clig_p)
    Ca20tnf=Ca20tnf_0*(2**Ca20tnf_p)
    Ca20lps=Ca20lps_0*(2**Ca20lps_p)
    aIRAK  =aIRAK_0*(2**aIRAK_p)
    dIRAK  =dIRAK_0*(2**dIRAK_p)
    dIRAKi =dIRAKi_0*(2**dIRAKi_p)
    mu     =mu_0*(2**mu_p)
    beta   =beta_0*(2**beta_p)
    dL     =dL_0*(2**dL_p)
    
    par = np.array([rNim,  KIc,  rIim, KIn,  trI,  KN,      dIm,     tlI,     aIKK,    trA,
                    dAm,   tlA,  dA,   aR,   Clig, Ca20tnf, Ca20lps, aIRAK,   dIRAK,   dIRAKi,
                    mu,    beta, dL])
    
    global ini
    args = (par, 1)
    # Integrate ODES for first ligand
    ini[11] = 0
    ini[12] = 0
    ini[13] = 1  #TNF:11, IL1:12, LPS:13
    t1 = np.linspace(0, t_max, 201)
    yz = scipy.integrate.odeint(NFkB_cascade, ini, t1, args=args)
    Nn1, Im1, I1, Am1, A1, IRAKlps1, IRAKilps1, IRAKil11, IRAKiil11, IKKa1, IKKi1, TNF1, IL11, LPS1 = yz.transpose()
    ini1 = yz[200,:]
    
    # Integrate ODES for second ligand
    ini1[11] = 0
    ini1[12] = 1
    ini1[13] = 0
    t2 = np.linspace(t_max, 2.5*t_max, 301)
    yz = scipy.integrate.odeint(NFkB_cascade, ini1, t2, args=args)
    Nn2, Im2, I2, Am2, A2, IRAKlps2, IRAKilps2, IRAKil12, IRAKiil12, IKKa2, IKKi2, TNF2, IL12, LPS2 = yz.transpose()
    
    t = np.concatenate([t1, t2])
    Nn = np.concatenate([Nn1, Nn2])
    Im = np.concatenate([Im1, Im2])
    I = np.concatenate([I1, I2])
    Am = np.concatenate([Am1, Am2])
    A = np.concatenate([A1, A2])
    IRAKlps = np.concatenate([IRAKlps1, IRAKlps2])
    IRAKilps = np.concatenate([IRAKilps1, IRAKilps2])
    IRAKil1 = np.concatenate([IRAKil11, IRAKil12])
    IRAKiil1 = np.concatenate([IRAKiil11, IRAKiil12])
    IKKa = np.concatenate([IKKa1, IKKa2])
    IKKi = np.concatenate([IKKi1, IKKi2])
    TNF = np.concatenate([TNF1, TNF2])
    IL1 = np.concatenate([IL11, IL12])
    LPS = np.concatenate([LPS1, LPS2])
    
    # Normalize Y and Z responses
    if normalize:
        Nn /= Nn.max()
        Im /= Im.max()
        I /= I.max()
        Am /= Am.max()
        A /= A.max()
        TNF /= TNF.max()
        IL1 /= IL1.max()
        LPS /= LPS.max()
    
    # Set up plot
    p2 = bokeh.plotting.figure(
         frame_width=300,
         frame_height=300,
         x_axis_label="Time (h)",
         y_axis_label=f"{'Normalized ' if normalize else ''}Normalized Level",
        y_range=[0, 1.2],
    )
    
    # Populate glyphs
    p2.line(t, Nn/.625, line_width=2, color=colors[0], legend_label="NF-kBnuc")
    p2.line(t, A/20,  line_width=2, color=colors[1], legend_label="Downstream Feedback")
    p2.line(t, (IRAKlps+IRAKil1)/1, line_width=2, color=colors[3], legend_label="Active IRAK")
    p2.line(t, (1-IRAKlps-IRAKilps-IRAKil1-IRAKiil1)/1, line_width=2, color=colors[4], legend_label="Available IRAK")
    p2.line(t, (1-IKKa-IKKi)/1, line_width=2, color=colors[5], legend_label="Available IKK")
    
    p2.axis.minor_tick_line_color = None
    p2.axis.major_tick_out = 5
    p2.axis.major_tick_in =  0
    p2.axis.axis_label_text_font_style = "normal"
    p2.grid.grid_line_color = None
    
    # Place the legend
    p2.legend.location = "top_right"
        
    p2.output_backend = 'svg'
    export_svg(p2, filename="p2.svg")
    
    return p2

@pn.depends(
    rNim_slider.param.value, KIc_slider.param.value,  rIim_slider.param.value, KIn_slider.param.value, trI_slider.param.value, KN_slider.param.value,  dIm_slider.param.value,
    tlI_slider.param.value,  aIKK_slider.param.value, trA_slider.param.value,  dAm_slider.param.value, tlA_slider.param.value, dA_slider.param.value,  aR_slider.param.value,
    Clig_slider.param.value, Ca20tnf_slider.param.value, Ca20lps_slider.param.value, aIRAK_slider.param.value, dIRAK_slider.param.value, dIRAKi_slider.param.value,
    mu_slider.param.value, beta_slider.param.value, dL_slider.param.value,
    t_max_slider.param.value, normalize_checkbox.param.value,
    )
def IL1_LPS_response_plot(
    rNim_p,  KIc_p,   rIim_p, KIn_p,  trI_p,  KN_p,      dIm_p,     tlI_p,     aIKK_p,    trA_p,
    dAm_p,   tlA_p,   dA_p,   aR_p,   Clig_p, Ca20tnf_p, Ca20lps_p, aIRAK_p,   dIRAK_p,   dIRAKi_p,
    mu_p,    beta_p,  dL_p,
    t_max,  normalize,
#     rNim_0,  KIc_0,    rIim_0, KIn_0,  trI_0,  KN_0,   dIm_0,     tlI_0,     aIKK_0,    dI_0,
#     trA_0,   dAm_0,    tlA_0,  dA_0,   aR_0,   Ctnf_0, Ca20tnf_0, Ca20il1_0, Ca20lps_0, aIRAK_0,
#     dIRAK_0, dIRAKi_0, mu_0,   beta_0, trIe_0, dIem_0, KDur_0,    tlIe_0,    dIe_0,     rNI_0,
#     dNI_0,   dL_0
    ):
    
    rNim=rNim_0*(2**rNim_p)
    KIc =KIc_0*(2**KIc_p)
    rIim=rIim_0*(2**rIim_p)
    KIn =KIn_0*(2**KIn_p)
    trI =trI_0*(2**trI_p)
    KN  =KN_0*(2**KN_p)
    dIm =dIm_0*(2**dIm_p)
    tlI =tlI_0*(2**tlI_p)
    aIKK=aIKK_0*(2**aIKK_p)
    trA =trA_0*(2**trA_p)
    dAm =dAm_0*(2**dAm_p)
    tlA =tlA_0*(2**tlA_p)
    dA  =dA_0*(2**dA_p)
    aR  =aR_0*(2**aR_p)
    Clig=Clig_0*(2**Clig_p)
    Ca20tnf=Ca20tnf_0*(2**Ca20tnf_p)
    Ca20lps=Ca20lps_0*(2**Ca20lps_p)
    aIRAK  =aIRAK_0*(2**aIRAK_p)
    dIRAK  =dIRAK_0*(2**dIRAK_p)
    dIRAKi =dIRAKi_0*(2**dIRAKi_p)
    mu     =mu_0*(2**mu_p)
    beta   =beta_0*(2**beta_p)
    dL     =dL_0*(2**dL_p)
    
    par = np.array([rNim,  KIc,  rIim, KIn,  trI,  KN,      dIm,     tlI,     aIKK,    trA,
                    dAm,   tlA,  dA,   aR,   Clig, Ca20tnf, Ca20lps, aIRAK,   dIRAK,   dIRAKi,
                    mu,    beta, dL])
    
    global ini
    args = (par, 1)
    # Integrate ODES for first ligand
    ini[11] = 0
    ini[12] = 0
    ini[13] = 1  #TNF:11, IL1:12, LPS:13
    t1 = np.linspace(0, t_max, 201)
    yz = scipy.integrate.odeint(NFkB_cascade, ini, t1, args=args)
    Nn1, Im1, I1, Am1, A1, IRAKlps1, IRAKilps1, IRAKil11, IRAKiil11, IKKa1, IKKi1, TNF1, IL11, LPS1 = yz.transpose()
    ini2 = yz[200,:]
    
    # Integrate ODES for second ligand
    ini2[11] = 0
    ini2[12] = 1
    ini2[13] = 0
    t2 = np.linspace(t_max, 2.5*t_max, 301)
    yz = scipy.integrate.odeint(NFkB_cascade, ini2, t2, args=args)
    Nn2, Im2, I2, Am2, A2, IRAKlps2, IRAKilps2, IRAKil12, IRAKiil12, IKKa2, IKKi2, TNF2, IL12, LPS2 = yz.transpose()
    
    t = np.concatenate([t1, t2])
    Nn = np.concatenate([Nn1, Nn2])
    Im = np.concatenate([Im1, Im2])
    I = np.concatenate([I1, I2])
    Am = np.concatenate([Am1, Am2])
    A = np.concatenate([A1, A2])
    IRAKlps = np.concatenate([IRAKlps1, IRAKlps2])
    IRAKilps = np.concatenate([IRAKilps1, IRAKilps2])
    IRAKil1 = np.concatenate([IRAKil11, IRAKil12])
    IRAKiil1 = np.concatenate([IRAKiil11, IRAKiil12])
    IKKa = np.concatenate([IKKa1, IKKa2])
    IKKi = np.concatenate([IKKi1, IKKi2])
    TNF = np.concatenate([TNF1, TNF2])
    IL1 = np.concatenate([IL11, IL12])
    LPS = np.concatenate([LPS1, LPS2])
    
    # Normalize Y and Z responses
    if normalize:
        Nn /= Nn.max()
        Im /= Im.max()
        I /= I.max()
        Am /= Am.max()
        A /= A.max()
        TNF /= TNF.max()
        IL1 /= IL1.max()
        LPS /= LPS.max()
    
    # Set up plot
    p3 = bokeh.plotting.figure(
         frame_width=300,
         frame_height=300,
         x_axis_label="Time (h)",
         y_axis_label=f"{'Normalized ' if normalize else ''}Normalized Level",
        y_range=[0, 1.2],
    )
    
    # Populate glyphs
    p3.line(t, Nn/.625, line_width=2, color=colors[0], legend_label="NF-kBnuc")
    p3.line(t, 4*A/20,  line_width=2, color=colors[1], legend_label="Downstream Feedback")
    p3.line(t, (IRAKlps+IRAKil1)/1, line_width=2, color=colors[3], legend_label="Active IRAK")
    p3.line(t, (1-IRAKlps-IRAKilps-IRAKil1-IRAKiil1)/1, line_width=2, color=colors[4], legend_label="Available IRAK")
    p3.line(t, (1-IKKa-IKKi)/1, line_width=2, color=colors[5], legend_label="Available IKK")
    
    p3.axis.minor_tick_line_color = None
    p3.axis.major_tick_out = 5
    p3.axis.major_tick_in =  0
    p3.axis.axis_label_text_font_style = "normal"
    p3.grid.grid_line_color = None
    
    # Place the legend
    # p3.legend.location = "top_right"
    
    p3.output_backend = 'svg'
    export_svg(p3, filename="p3.svg")
    
    return p3

In [4]:
widgets1 = pn.Column(
    pn.Spacer(height=10),
    rNim_slider,
    KIc_slider,
    rIim_slider,
    KIn_slider,
    trI_slider,
    KN_slider,
    dIm_slider,
    tlI_slider,
    aIKK_slider,
    trA_slider,
    width=200,    
)
widgets2 = pn.Column(
    pn.Spacer(height=10),
    dAm_slider,
    tlA_slider,
    dA_slider,
    aR_slider,
    Clig_slider,
    Ca20tnf_slider,
    Ca20lps_slider,
    aIRAK_slider,
    dIRAK_slider,
    dIRAKi_slider,
    width=200,    
)
widgets3 = pn.Column(
    pn.Spacer(height=10),
    mu_slider,
    beta_slider,
    dL_slider,
    t_max_slider,
    width=200,    
)
# ^ in the order of rNim,  KIc,    rIim, KIn,  trI,  KN,   dIm,     tlI,     aIKK,    dI,
#                   trA,   dAm,    tlA,  dA,   aR,   Clig, Ca20tnf, Ca20il1, Ca20lps, aIRAK, 
#                   dIRAK, dIRAKi, mu,   beta, trIe, dIem, KDur,    tlIe,    dIe,     rNI,
#                   dNI,   dL

left_column = pn.Column(
    pn.Row(pn.Spacer(width=20), normalize_checkbox),  initial_response_plot,
    LPS_IL1_response_plot, IL1_LPS_response_plot,
)

# Final layout
pn.Row(left_column, pn.Spacer(width=10), widgets1, pn.Spacer(width=10), widgets2, pn.Spacer(width=10), widgets3)

Row
    [0] Column
        [0] Row
            [0] Spacer(width=20)
            [1] Checkbox(name='Normalize')
        [1] ParamFunction(function)
        [2] ParamFunction(function)
        [3] ParamFunction(function)
    [1] Spacer(width=10)
    [2] Column(width=200)
        [0] Spacer(height=10)
        [1] FloatSlider(end=5, name='rNim_p', start=-5)
        [2] FloatSlider(end=5, name='KIc_p', start=-5)
        [3] FloatSlider(end=5, name='rIim_p', start=-5)
        [4] FloatSlider(end=5, name='KIn_p', start=-5)
        [5] FloatSlider(end=5, name='trI_p', start=-5)
        [6] FloatSlider(end=5, name='KN_p', start=-5)
        [7] FloatSlider(end=5, name='dIm_p', start=-5)
        [8] FloatSlider(end=5, name='tlI_p', start=-5)
        [9] FloatSlider(end=5, name='aIKK_p', start=-5)
        [10] FloatSlider(end=5, name='trA_p', start=-5)
    [3] Spacer(width=10)
    [4] Column(width=200)
        [0] Spacer(height=10)
        [1] FloatSlider(end=5, name='dAm_p', start=-5)
        [2] FloatSlider(end=5, name='tlA_p', start=-5)
        [3] FloatSlider(end=5, name='dA_p', start=-5, value=-2)
        [4] FloatSlider(end=5, name='aR_p', start=-5)
        [5] FloatSlider(end=5, name='Clig_p', start=-5, value=-0.4)
        [6] FloatSlider(end=5, name='Ca20tnf_p', start=-5)
        [7] FloatSlider(end=5, name='Ca20lps_p', start=-5)
        [8] FloatSlider(end=5, name='aIRAK_p', start=-5)
        [9] FloatSlider(end=5, name='dIRAK_p', start=-5)
        [10] FloatSlider(end=5, name='dIRAKi_p', start=-5)
    [5] Spacer(width=10)
    [6] Column(width=200)
        [0] Spacer(height=10)
        [1] FloatSlider(end=5, name='mu_p', start=-5, value=0.5)
        [2] FloatSlider(end=5, name='beta_p', start=-5)
        [3] FloatSlider(end=5, name='dL_p', start=-5)
        [4] FloatSlider(end=8, name='t_max', start=0.1, value=2)